<a href="https://colab.research.google.com/github/Posthumanlight/counterpick.gg/blob/main/counterpick_gg_test_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#виносимо всі імпорти на початок, щоб адекватно контролювати dependencies

import requests
import json

import numpy as np
import pandas as pd

from google.colab import drive
from google.colab import auth
from google.cloud import bigquery
import pandas_gbq


In [ ]:
# входимо і вписуємо апішні ключі, налаштовуємо сесію

auth.authenticate_user()
api_key = "RGAPI-2f8fdec1-e711-49eb-9098-54c890ea7d3d"
drive.mount('/content/drive')

bq_project_id = 'poetic-inkwell-412714'
bq_client = bigquery.Client(project=bq_project_id)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Функції для подальшого використання

#трансформуємо списки в датафрейми і завантажуємо їх в бігквері
def set_match_id_in_new_list(list, match_id):
  for i in range(len(list)):
    list[i].append([match_id])
def load_table_list2bq(list, column_names, table_bq, transformation = None):
  df_from_list=pd.DataFrame(list).set_axis(column_names, axis = 'columns', copy = False)
  df_from_list['match_id'] = df_from_list['match_id'].astype(str)
  match transformation:
    case 'string':
      df_from_list = df_from_list.astype(str)

  pandas_gbq.to_gbq(dataframe = df_from_list, destination_table = table_bq, project_id = bq_project_id , if_exists='append')

In [ ]:
gameName = input('Nickname ')
tag_line = input('Tag ')


Nickname PepegaKiller
Tag SS


In [ ]:
summoner_by_name = "https://europe.api.riotgames.com/riot/account/v1/accounts/by-riot-id/" + gameName + "/" + tag_line + "?api_key=" + api_key

In [ ]:
requests.get(summoner_by_name)

<Response [200]>

In [ ]:
resp = requests.get(summoner_by_name)

In [ ]:
player_info = resp.json()
player_puuid = player_info['puuid']


In [ ]:
last_20match_id_by_puuid_url = "https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/" + player_puuid + "/ids?start=0&count=20&api_key=" + api_key
requests.get(last_20match_id_by_puuid_url)
resp_last_20match_id_by_puuid_url = requests.get(last_20match_id_by_puuid_url)
last_20match_id_by_puuid = resp_last_20match_id_by_puuid_url.json()
last_20match_id_by_puuid

['EUW1_6811027417',
 'EUW1_6810952380',
 'EUW1_6806267824',
 'EUW1_6806215972',
 'EUW1_6803391247',
 'EUW1_6803301902',
 'EUW1_6798759960',
 'EUW1_6798700711',
 'EUW1_6798626936',
 'EUW1_6798554464',
 'EUW1_6798495414',
 'EUW1_6798437132',
 'EUW1_6798388751',
 'EUW1_6798335898',
 'EUW1_6798280561',
 'EUW1_6797489801',
 'EUW1_6797448605',
 'EUW1_6797356433',
 'EUW1_6797278763',
 'EUW1_6797209380']

In [ ]:
list_match_metadata = []
list_player_id = []
list_champion_id= []
list_kill_per_champion = []
list_death_per_champion = []
list_assist_per_champion = []
list_gpm_per_champion = []
list_dpm_per_champion = []

for i in range(20):
  #отримуємо і формуємо джейсони відповідей
  match_info_by_match_id_url = "https://europe.api.riotgames.com/lol/match/v5/matches/" + last_20match_id_by_puuid[i] + "?api_key=" + api_key
  resp_match_info_by_match_id_url = requests.get(match_info_by_match_id_url)
  match_info_by_match_id = resp_match_info_by_match_id_url.json()
  match_timeline_by_match_id_url = "https://europe.api.riotgames.com/lol/match/v5/matches/" + last_20match_id_by_puuid[i] + '/timeline' + "?api_key=" + api_key
  match_timeline_by_match_id = requests.get(match_timeline_by_match_id_url).json()

  #Ініціалізовуємо списки різних атрибутів


  #Заповнюємо лісти даними з джейсона
  list_match_metadata.append(match_info_by_match_id['info']['gameId'])
  list_match_metadata.append(match_info_by_match_id['info']['participants'][0]['win'])
  list_match_metadata.append(round(match_info_by_match_id['info']['gameDuration']))
  if list_match_metadata[1] == 'true':
    list_match_metadata[1] = 1
  else:
    list_match_metadata[1] = 0
  set_match_id_in_new_list([list_player_id, list_champion_id, list_kill_per_champion, list_death_per_champion, list_assist_per_champion, list_gpm_per_champion, list_dpm_per_champion], match_info_by_match_id['info']['gameId'])
  for j in range(10):
    list_player_id[i].append(match_info_by_match_id['metadata']['participants'][j])
    list_champion_id[i].append(match_info_by_match_id['info']['participants'][j]['championId'])
    list_kill_per_champion[i].append(match_info_by_match_id['info']['participants'][j]['kills'])
    list_death_per_champion[i].append(match_info_by_match_id['info']['participants'][j]['deaths'])
    list_assist_per_champion[i].append(match_info_by_match_id['info']['participants'][j]['assists'])
    list_gpm_per_champion[i].append(round(match_info_by_match_id['info']['participants'][j]['challenges']['goldPerMinute']))
    list_dpm_per_champion[i].append(match_info_by_match_id['info']['participants'][j]['challenges']['damagePerMinute'])

  #Завантажуємо дані в бігквері
load_table_list2bq(list_match_metadata, ['match_id', 'match_win_team1', 'match_duration'], 'match_information.match_information_metadata')
load_table_list2bq(list_player_id, ['match_id', 'player_id1','player_id2','player_id3','player_id4','player_id5','player_id6','player_id7','player_id8','player_id9','player_id10'], 'match_information.match_information_players')
load_table_list2bq(list_champion_id, ['match_id', 'champion_id1','champion_id2','champion_id3','champion_id4','champion_id5','champion_id6','champion_id7','champion_id8','champion_id9','champion_id10'], 'match_information.match_information_champions_picked', 'string')
load_table_list2bq(list_kill_per_champion, ['match_id', 'champion_kill1','champion_kill2','champion_kill3','champion_kill4','champion_kill5','champion_kill6','champion_kill7','champion_kill8','champion_kill9','champion_kill10'], 'match_information.match_information_champions_kill')
load_table_list2bq(list_death_per_champion, ['match_id', 'champion_death1','champion_death2','champion_death3','champion_death4','champion_death5','champion_death6','champion_death7','champion_death8','champion_death9','champion_death10'], 'match_information.match_information_champions_death')
load_table_list2bq(list_assist_per_champion, ['match_id', 'champion_assist1','champion_assist2','champion_assist3','champion_assist4','champion_assist5','champion_assist6','champion_assist7','champion_assist8','champion_assist9','champion_assist10'], 'match_information.match_information_champions_assist')
load_table_list2bq(list_gpm_per_champion, ['match_id', 'champion_gpm1','champion_gpm2','champion_gpm3','champion_gpm4','champion_gpm5','champion_gpm6','champion_gpm7','champion_gpm8','champion_gpm9','champion_gpm10'], 'match_information.match_information_champions_gpm')
load_table_list2bq(list_dpm_per_champion, ['match_id', 'champion_dpm1','champion_dpm2','champion_dpm3','champion_dpm4','champion_dpm5','champion_dpm6','champion_dpm7','champion_dpm8','champion_dpm9','champion_dpm10'], 'match_information.match_information_champions_dpm')


ValueError: Length mismatch: Expected axis has 1 elements, new values have 3 elements

In [ ]:
'''with open('drive/MyDrive/match_data_json_test.json', 'w') as f:
  json.dump(match_info_by_match_id, f)
with open('drive/MyDrive/match_timeline_json_test.json', 'w') as f:
  json.dump(match_timeline_by_match_id, f)
'''